In [1]:
# ThermostatSC2

In [2]:
import datetime
import logging
import random
import time
import uuid

import smartpynector as sp
from utils import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ThermostatSC")

In [3]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
# prefixes of ontologies used in the graph patterns
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
}
# temp measurement graph pattern
TEMP_MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id ."""
# graph pattern describing the change of the ?desired_temp setting of the thermostat
TEMP_SETTING_GRAPH_PATTERN = """?setting rdf:type saref:SetLevelCommand .
                            ?setting saref:hasValue ?desired_temp .
                            ?setting saref:isMeasuredIn saref:TemperatureUnit .
                            ?setting saref:hasTimestamp ?timestamp .
                            ?setting saref:isCommandOf ?room_id .
                            ?setting saref:relatesToProperty saref:Temperature .
                            ?setting saref:commandIssuedBy ?device_id .
                            """
# humidity measurement graph pattern
HUMIDITY_MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?humidity .
                        ?meas saref:isMeasuredIn saref:HumidityUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Humidity .
                        ?meas saref:measurementMadeBy ?device_id ."""
# graph pattern describing the change of the ?desired_humidity setting of the thermostat
HUMIDITY_SETTING_GRAPH_PATTERN = """?setting rdf:type saref:SetLevelCommand .
                            ?setting saref:hasValue ?desired_humidity .
                            ?setting saref:isMeasuredIn saref:HumidityUnit .
                            ?setting saref:hasTimestamp ?timestamp .
                            ?setting saref:isCommandOf ?room_id .
                            ?setting saref:relatesToProperty saref:Humidity .
                            ?setting saref:commandIssuedBy ?device_id .
                            """

In [4]:
def get_temp_now():
    return requests.get(THERMOSTAT_API_URL + "/actual_temperature").json()

In [5]:
def temp_measurement_loop(post_temp_ki_id, kb_id, ke_endpoint):
    value = get_temp_now()
    logger.info(
        f"attempting to post temp measurement of {value} units at {sp.get_timestamp_now()}"
    )
    post(
        [
            {
                "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
                "temp": f"{value}",
                "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
                "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
            }
        ],
        post_temp_ki_id,
        kb_id,
        ke_endpoint,
    )
    logger.info(
        f"Published temperature measurement of {value} units at {sp.get_timestamp_now()}"
    )

    time.sleep(5)

In [6]:
def get_humidity_now():
    return requests.get(THERMOSTAT_API_URL + "/actual_humidity").json()

In [7]:
def humidity_measurement_loop(post_humidity_ki_id, kb_id, ke_endpoint):
    value = get_humidity_now()
    logger.info(
        f"attempting to post humidity measurement of {value} units at {sp.get_timestamp_now()}"
    )

    post(
        [
            {
                "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
                "humidity": f"{value}",
                "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
                "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
            }
        ],
        post_humidity_ki_id,
        kb_id,
        ke_endpoint,
    )
    logger.info(
        f"Published humidity measurement of {value} units at {sp.get_timestamp_now()}"
    )

    time.sleep(5)

In [8]:
def handle_react_change_desired_temp(bindings):
    for binding in bindings:
        logger.info(f"Changing desired temperature to: {binding['desired_temp']}")
        # change desired temp
        requests.put(
            THERMOSTAT_API_URL
            + "/desired_temperature"
            + f'?desired_temperature={int(binding["desired_temp"])}'
        )
    return []

In [9]:
def handle_react_change_desired_humidity(bindings):
    for binding in bindings:
        logger.info(f"Changing desired humidity to: {binding['desired_humidity']}")
        # change desired temp
        requests.put(
            THERMOSTAT_API_URL
            + "/desired_humidity"
            + f'?desired_humidity={int(binding["desired_humidity"])}'
        )
    return []

In [10]:
import threading


def start_sensor_kb(
    kb_id, kb_name, kb_description, ke_endpoint, post_live_measurements=True
):
    # delete in case allready exists
    delete_knowledge_base(kb_id, ke_endpoint)

    # register kb
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    post_temp_ki_id = register_post_knowledge_interaction(
        TEMP_MEAS_GRAPH_PATTERN,
        None,
        "post-temp-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    post_humidity_ki_id = register_post_knowledge_interaction(
        HUMIDITY_MEAS_GRAPH_PATTERN,
        None,
        "post-humidity-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    temp_react_ki_id = register_react_knowledge_interaction(
        TEMP_SETTING_GRAPH_PATTERN,
        None,
        "set-desired-temp",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    humidity_react_ki_id = register_react_knowledge_interaction(
        HUMIDITY_SETTING_GRAPH_PATTERN,
        None,
        "set-desired-humidity",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    def measurement_loop(post_temp_ki_id, post_humidity_ki_id, kb_id, ke_endpoint):
        while True:
            temp_measurement_loop(post_temp_ki_id, kb_id, ke_endpoint)
            humidity_measurement_loop(post_humidity_ki_id, kb_id, ke_endpoint)

    measurement_thread = threading.Thread(
        target=measurement_loop,
        args=(
            post_temp_ki_id,
            post_humidity_ki_id,
            kb_id,
            ke_endpoint,
        ),
    )

    if post_live_measurements:
        measurement_thread.start()

    # Start the start_handle_loop function in a separate thread
    start_handle_loop(
        {
            temp_react_ki_id: handle_react_change_desired_temp,
            humidity_react_ki_id: handle_react_change_desired_humidity,
        },
        kb_id,
        ke_endpoint,
    )

In [11]:
start_sensor_kb(
    "http://example.org/thermostatSC",
    "Thermostat_1",
    "A thermostat",
    "http://knowledge_engine:8280/rest/",
)

2023-06-23 16:56:05 INFO deleted http://example.org/thermostatSC
2023-06-23 16:56:05 INFO registered Thermostat_1
2023-06-23 16:56:05 INFO received issued knowledge interaction id: http://example.org/thermostatSC/interaction/post-temp-measurements
2023-06-23 16:56:05 INFO received issued knowledge interaction id: http://example.org/thermostatSC/interaction/post-humidity-measurements
2023-06-23 16:56:05 INFO received issued knowledge interaction id: http://example.org/thermostatSC/interaction/set-desired-temp
2023-06-23 16:56:05 INFO received issued knowledge interaction id: http://example.org/thermostatSC/interaction/set-desired-humidity
2023-06-23 16:56:05 INFO attempting to post temp measurement of 18 units at 2023-06-23T16:56:05+00:00
2023-06-23 16:56:05 INFO Published temperature measurement of 18 units at 2023-06-23T16:56:05+00:00
2023-06-23 16:56:10 INFO attempting to post humidity measurement of 51 units at 2023-06-23T16:56:10+00:00
2023-06-23 16:56:10 INFO Published humidity me

KeyboardInterrupt: 